<a href="https://colab.research.google.com/github/Drishti9/speechEmotionDetection/blob/master/EmotionAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install librosa soundfile 

!apt-get install -q portaudio19-dev python3-pyaudio
!pip install pysine
!pip install pyaudio

In [13]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [14]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

observed_emotions=['calm', 'happy', 'fearful', 'disgust']

def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/ravdess-data/Actor_*//*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [15]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [16]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy of MLP Classifier: {:.2f}%".format(accuracy*100))

Accuracy of MLP Classifier: 70.47%


In [17]:
from sklearn.ensemble import RandomForestClassifier

model2=RandomForestClassifier(max_depth=7, random_state=0)
model2.fit(x_train,y_train)
y_pred=model2.predict(x_test)

accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy of Random Forest Classi: {:.2f}%".format(accuracy*100))

Accuracy of Random Forest Classi: 65.28%


In [18]:
from sklearn.neighbors import KNeighborsClassifier

model5= KNeighborsClassifier(n_neighbors=3)
model5.fit(x_train,y_train)
y_pred=model5.predict(x_test)

accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 67.36%


In [19]:
def pred_audio(file):
    
    file_name=os.path.basename(file)
    emotion_true=emotions[file_name.split("-")[2]]
    # if emotion_true not in observed_emotions:
    #       return emotion_true, "cant predict"
    feature=extract_feature(file, mfcc=True, chroma=True, mel=True).reshape(1,-1)
    emotion_pred=model.predict(feature)
    return emotion_true, emotion_pred

true_val, pred_val=pred_audio("/content/drive/MyDrive/ravdess-data/Actor_11/03-01-02-02-01-02-11.wav")
print("True emotion: ", true_val)
print("Predicted Emotion: ", pred_val)

True emotion:  calm
Predicted Emotion:  ['calm']
